# 05 Hittable object - Numpy broadcasting

* 기존 예제에서 for loop을 제거하고 Numpy의 broadcasting을 이용하여 가속한 예제입니다.
* GPU를 사용하는 Pytorch 등으로 쉽게 치환할 수 있고 더욱 빨라집니다.

In [25]:
from __future__ import annotations

from typing import List
import sys

import numpy as np
import numpy.linalg as LA

import matplotlib.pyplot as plt

infinity = sys.float_info.max

In [26]:
def normalize(vec: np.array, eps: float = 1e-6):
    return vec / (LA.norm(vec, axis=-1, keepdims=True) + eps)


def blend(color1: np.array, color2: np.array, t_map: np.array):
    return (1.0 - t_map) * color1 + t_map * color2


def dot(v1: np.array, v2: np.array) -> np.array:
    return np.sum(v1 * v2, axis=-1, keepdims=True)

In [27]:
class Ray:
    def __init__(self, origin: np.array, direction_map: np.array) -> None:
        self.origin = origin
        self.direction_map = normalize(direction_map)
        self.height, self.width = self.direction_map.shape[:2]

    def at(self, t_map: np.array) -> np.array:
        return self.origin + t_map * self.direction_map

# Hittable과 HitRecord

Numpy를 이용해서 Hittable과 HitRecord를 구현합니다.
* Hittable은 기존 구현과 입력의 타입이 np.array인 것 외에는 큰 차이가 없습니다.
* HitRecord에는 valid_map을 추가 합니다.
    * valid_map은 각 ray가 물체와 만났는지를 의미합니다.
    * update는 아래의 HittableList에서 사용되는 함수로 가장 가까운 물체의 Hit 정보로 갱신하는데 사용됩니다.

In [28]:
class HitRecord:
    def __init__(
        self,
        point_map: np.array,
        normal_map: np.array,
        t_map: np.array,
        valid_map: np.array,
    ) -> None:
        self.point_map = point_map
        self.normal_map = normal_map
        self.t_map = t_map
        self.valid_map = valid_map

    def set_face_normal(self, ray: Ray, outward_normal_map: np.array) -> None:
        is_front_face = dot(ray.direction_map, outward_normal_map) < 0.0
        self.normal_map = np.where(
            is_front_face, outward_normal_map, -outward_normal_map
        )

    def update(self, other: HitRecord) -> None:
        self.point_map = np.where(other.valid_map, other.point_map, self.point_map)
        self.normal_map = np.where(other.valid_map, other.normal_map, self.normal_map)
        self.t_map = np.where(other.valid_map, other.t_map, self.t_map)
        self.valid_map = np.where(other.valid_map, other.valid_map, self.valid_map)

In [29]:
class Hittable:
    def hit(self, ray: Ray, t_min_map: np.array, t_max_map: np.array) -> HitRecord:
        raise NotImplementedError

In [30]:
class Sphere(Hittable):
    def __init__(self) -> None:
        self.center = np.zeros(3).reshape([1, 1, 3])
        self.radius = 1.0

    def __init__(self, center: np.array, radius: float) -> None:
        self.center = center.reshape([1, 1, 3])
        self.radius = radius

    def hit(self, ray: Ray, t_min_map: np.array, t_max_map: np.array) -> bool:
        dir_center_to_origin = ray.origin - self.center

        a_map = dot(ray.direction_map, ray.direction_map)
        half_b_map = dot(dir_center_to_origin, ray.direction_map)
        c_map = dot(dir_center_to_origin, dir_center_to_origin) - self.radius**2.0

        discriminant_map = half_b_map**2 - a_map * c_map
        cond_discriminant = discriminant_map >= 0.0

        safe_discriminant_map = np.where(cond_discriminant, discriminant_map, 0.0)

        sqrt_d_map = np.sqrt(safe_discriminant_map)

        # find the nearest root that lies in the acceptable range.
        t_map1 = (-half_b_map - sqrt_d_map) / a_map
        cond1 = np.logical_and(t_min_map <= t_map1, t_map1 <= t_max_map)
        t_map2 = (-half_b_map + sqrt_d_map) / a_map
        cond2 = np.logical_and(t_min_map <= t_map2, t_map2 <= t_max_map)

        valid_map = np.logical_and(cond_discriminant, np.logical_or(cond1, cond2))

        t_map = np.where(cond1, t_map1, t_map2)

        point_map = ray.at(t_map)
        outward_normal_map = (point_map - self.center) / self.radius
        hit_record = HitRecord(
            point_map=point_map,
            normal_map=outward_normal_map,
            t_map=t_map,
            valid_map=valid_map,
        )
        hit_record.set_face_normal(ray=ray, outward_normal_map=outward_normal_map)
        return hit_record

HittableList도 이해와 확장성을 위해 기존 코드와 유사하게 구현합니다.
* valid_map을 이용해서 ray가 만나는 가장 가까운 물체의 가장 먼 지점의 거리를 갱신합니다.
* record의 각 ray에 해당하는 정보는 update로 갱신합니다.

In [31]:
class HittableList(Hittable):
    def __init__(self) -> None:
        self.objects: List[Hittable] = []

    def clear(self) -> None:
        self.objects.clear()

    def add(self, object: Hittable) -> None:
        self.objects.append(object)

    def hit(self, ray: Ray, t_min_map: float, t_max_map: float) -> HitRecord:
        record = None
        closest_so_far_map = t_max_map

        for object in self.objects:
            tmp_record = object.hit(
                ray=ray, t_min_map=t_min_map, t_max_map=closest_so_far_map
            )
            closest_so_far_map = np.where(
                tmp_record.valid_map, tmp_record.t_map, closest_so_far_map
            )
            if record is None:
                record = tmp_record
            else:
                record.update(tmp_record)

        return record

* ray_color는 모든 ray에 대해서
    * ray가 장면의 물체들과 만났을때 표면의 normal 색상과 배경 색상을 모두 계산하고
    * ray가 물체를 만났는지를 이용해서 최종 색상을 결정합니다.

In [32]:
def ray_color(ray: Ray, world: Hittable) -> np.array:
    record = world.hit(ray=ray, t_min_map=0, t_max_map=infinity)

    world_color = 0.5 * (record.normal_map + 1.0)

    t_map = 0.5 * (ray.direction_map[..., 1:2] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    background_color = blend(color1=color1, color2=color2, t_map=t_map)

    return np.where(record.valid_map, world_color, background_color)

In [41]:
# image
aspect_ratio = 16.0 / 9.0
image_width = 400
image_height = int(image_width / aspect_ratio)

두 개의 구로 장면을 구성합니다.

In [42]:
# world
world = HittableList()
world.add(Sphere(center=np.array([0.0, 0.0, -1.0]), radius=0.5))
world.add(Sphere(center=np.array([0.0, -100.5, -1.0]), radius=100.0))

이후의 구현은 이전과 동일합니다.

In [43]:
# camera
# viewport_height = 2.0
# viewport_width = aspect_ratio * viewport_height
viewport_height = 2.0
viewport_width = aspect_ratio * viewport_height
focal_length = 1.0

camera_origin = np.array([0.0, 0.0, 0.0])
horizontal_vec = np.array([viewport_width, 0.0, 0.0])
vertical_vec = np.array([0.0, viewport_height, 0.0])
frontal_vec = np.array([0.0, 0.0, focal_length])
lower_left_corner = (
    camera_origin - horizontal_vec / 2.0 - vertical_vec / 2.0 - frontal_vec
)

pixel_delta_u = horizontal_vec / image_width
pixel_delta_v = vertical_vec / image_height

In [ ]:
%%time

us = np.arange(start=0, stop=image_width) + 0.5
vs = np.arange(start=0, stop=image_height) + 0.5
u_map, v_map = np.meshgrid(us, vs, indexing="xy")
u_map = u_map[..., np.newaxis]
v_map = v_map[..., np.newaxis]

ray = Ray(origin=camera_origin,
          direction_map=(lower_left_corner + u_map*pixel_delta_u + v_map*pixel_delta_v) - camera_origin)

image = ray_color(ray=ray, world=world)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()

In [ ]:
plt.imshow(image[60:90, 135:165], origin="lower")
plt.show()